# Model Example

In this initial experiment, we will attempt to predict the diagnosis of ASD from fMRI data using Graph Neural Networks. Each sample in the dataset corresponds to an fMRI exam, and from that exam, we construct a graph using a brain atlas reference, which will be the input for the GNN.

# Constructing the full set of graphs

An input graph to a GNN consists on a triple $G = (V, E, X)$, where $V$ is the set of notes, $E$ is the set of edges and $X$ is the feature matrix such that the $i$-th row of $X$ contains the feature vector of node $i$.

For the set $V$, the nodes of the graph will correspond to Regions Of Interest (ROI), where a region of interest is considered to have an important and specialized function in the brain connectivity. These regions can be categorized using a brain atlas, and here, we will use the AAL atlas, with 116 ROIs.

The set of edges $E$ is calculated using the functional connectivity of the brain. In an fMRI exam, each part of the brain has a BOLD signal, a time series that measures brain activity through the presence of blood. Edges in the graph are inserted for pairs of nodes that have high positive correlation in their correspondent BOLD signals.

Finally, inspired by the work of [Zhu et al.](https://link.springer.com/chapter/10.1007/978-3-030-93049-3_30), $X$ will be equal to the correlation matrix of the BOLD signals for all regions of interest. Therefore, the feature vector $i$ corresponds to it's partial correlations to every other region of interest in the brain.



## Import libraries

In [ ]:
import networkx as nx
import numpy as np
import pandas as pd
import torch

In [ ]:
torch_version = str(torch.__version__)
scatter_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
sparse_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
!pip install torch-scatter -f $scatter_src
!pip install torch-sparse -f $sparse_src
!pip install torch-geometric

Looking in links: https://pytorch-geometric.com/whl/torch-2.5.1+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 33.6 MB/s eta 0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch-2.5.1+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.0 MB/s eta 0:00:00


In [ ]:
torch.set_default_tensor_type(torch.cuda.FloatTensor)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.10/dist-packages/torch/__init__.py:1144: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:432.)
  _C._set_default_tensor_type(t)


In [ ]:
def torch_pearson_correlation_matrix(X):

  mean = torch.mean(X, dim=0)
  covariance = torch.matmul((X - mean).t(), (X - mean))
  variance = torch.diagonal(covariance)
  correlation = (variance**(-1/2)) * covariance * ((variance**(-1/2)).t())
  return correlation

def construct_graph(roi_time_series, threshold=0.7, return_functional_connectivity=True, adjacency_matrix_type='torch', adjacency_representation = 'edge_list'):

  roi_time_series = torch.from_numpy(roi_time_series).to(device)
  functional_connectivity = torch_pearson_correlation_matrix(roi_time_series)
  adjacency = (functional_connectivity > threshold).int().to(device) - torch.eye(len(functional_connectivity))

  if adjacency_representation == 'edge_list':
    adjacency = (adjacency.nonzero()).t()

  if adjacency_matrix_type == 'numpy':
    adjacency = adjacency.cpu().numpy()

  if return_functional_connectivity==True:
    return adjacency, functional_connectivity
  else:
    return adjacency

Obtaining data from the ABIDE I [Preprocessed Connectomes Project](http://preprocessed-connectomes-project.org/abide/download.html), where all fMRI exams have been pre-processed with different brain atlases. As mentioned before, we will use the AAL atlas.

In [ ]:
import os
import subprocess
from torch_geometric.data import Data

def download_sample(sample_name):
  destination_folder = '/content/data/fmri'
  url = 'https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Outputs/cpac/filt_global/rois_aal/{}_rois_aal.1D'.format(sample_name)
  subprocess.run([
    "wget",
    "-P", destination_folder,
    "--continue",
    "--quiet",
    "--no-check-certificate",
    url
  ])

def get_dataset(threshold=0.7, download=True):

  if download==True:
    os.makedirs('/content/data/fmri', exist_ok=True)
    !wget -P /content/data https://raw.githubusercontent.com/preprocessed-connectomes-project/abide/master/Phenotypic_V1_0b_preprocessed1.csv
    phenotypic_data = pd.read_csv('/content/data/Phenotypic_V1_0b_preprocessed1.csv')
    phenotypic_data['FILE_ID'].apply(download_sample)

  phenotypic_data = pd.read_csv('/content/data/Phenotypic_V1_0b_preprocessed1.csv') #Apagar

  data_list = []
  for index, row in phenotypic_data.iterrows():
    filepath = "/content/data/fmri/" + row['FILE_ID'] + "_rois_aal.1D"
    if not os.path.exists(filepath): #Not every line of the phenotypic table is in the PCP dataset
      continue
    roi_time_series = np.loadtxt(filepath)
    edge_index, x = construct_graph(roi_time_series=roi_time_series, threshold=threshold)
    y = torch.tensor([row['DX_GROUP']-1]) #For the graph labels, 0 is for autistic patients and 1 is for typical control (adjusted to 0-1 interval to allow BCE Loss)
    data_list.append(Data(x=x, edge_index=edge_index, y=y))

  return data_list

In [ ]:
dataset = get_dataset()

In [ ]:
from torch.utils.data import random_split
from torch import Generator

train_size = int(0.8 * len(dataset)) #80% of the dataset goes into the training set
test_size = len(dataset) - train_size

generator = Generator(device='cuda')
train_dataset, test_dataset = random_split(dataset, [train_size, test_size], generator=generator)

# Training the GNN

Here, we will use the Graph Attention Network architecture proposed by [Veličković et al.](https://arxiv.org/abs/1710.10903)

In [ ]:
from torch.nn import ModuleList, Sigmoid
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn.conv import GATConv
from torch_geometric.nn.pool import global_mean_pool

class GAT(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim=15, n_heads=4, num_layers=3, dropout=0.5):

    super(GAT, self).__init__()
    self.dropout = dropout

    self.convs = ModuleList()
    self.convs.append(GATConv(input_dim, hidden_dim, n_heads))
    for l in range(num_layers - 1):
      self.convs.append(GATConv(hidden_dim * n_heads, hidden_dim, n_heads))

    self.dropout = dropout
    self.post_message_passing = nn.Linear(hidden_dim * n_heads, 1)
    self.sigmoid = Sigmoid()

  def forward(self, data):

    x, edge_index, batch = data.x, data.edge_index, data.batch

    for i in range(self.num_layers):
      x = self.convs[i](x, edge_index)
      x = F.relu(x)
      x = F.dropout(x, p=self.dropout,training=self.training)

    x = global_mean_pool(x, batch)
    x = self.post_message_passing(x)
    out = self.sigmoid(x)
    return out

In [ ]:
from torch.nn.functional import binary_cross_entropy as BCELoss
from torch.optim import Adam

def train(train_loader, n_epochs=1000, learning_rate = 0.01, return_losses = False):

  model = GAT(input_dim=116)
  optimizer = Adam(model.parameters(), lr=learning_rate)

  losses = []

  model.train()
  for epoch in range(n_epochs):

    total_loss = 0
    for batch in train_loader:
      optimizer.zero_grad()
      prediction = model(batch)
      true_label = batch.y
      loss = BCELoss(prediction, true_label)
      loss.backward()
      optimizer.step()
      total_loss += loss.item() * batch.num_graphs

    losses.append(total_loss)

  if return_losses:
    return model, losses
  return model

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metric import classification_report

def test(model, test_loader, losses):

  model.eval()

  test_prediction = []
  test_true_label = []

  for batch in test_loader:

    output = model(batch)
    y_batch_pred = torch.round(output)
    test_prediction.extend(y_batch_pred.cpu().numpy())
    test_true_label.extend(batch.y.cpu().numpy())

  print("Classification Report:")
  print(classification_report(test_true_label, test_prediction))

  plt.title("Decay of loss during training")
  plt.plot(losses)
  plt.show()

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)
model, losses = train(train_loader, return_losses=True)

test(model, test_loader, losses)